- Run Zookeeper
    - .\bin\windows\zookeeper-server-start.bat .\config\zookeeper.properties

- Start Kafka Server
    - .\bin\windows\kafka-server-start.bat .\config\server.properties

- Create Topics 
    - .\bin\windows\kafka-topics.bat --create --topic test --bootstrap-server localhost:9092

- Consume Topics 
    - .\bin\windows\kafka-console-consumer.bat --topic test --from-beginning --bootstrap-server localhost:9092


In [2]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from json import dumps
import json
from kafka import KafkaProducer
from json import dumps



In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
from nltk.corpus import stopwords

import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sklearn.decomposition import LatentDirichletAllocation

from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [4]:
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: dumps(x).encode('utf-8'))



In [5]:
#producer.send('demo_test', value="{'name': 'Consumer is was working well'}")

In [6]:
#df = pd.read_csv("indexProcessed.csv")
IMDB_df = pd.read_csv('https://raw.githubusercontent.com/manojjha/BITS_Assignments/main/NLP/IMDB%20Dataset.csv?token=GHSAT0AAAAAACJXQMXDNSKYL6LUDAUK5SISZMICSXQ')
IMDB_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
'''while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('test', value=dict_stock)
    '''

'while True:\n    dict_stock = df.sample(1).to_dict(orient="records")[0]\n    producer.send(\'test\', value=dict_stock)\n    '

In [8]:
IMDB_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [9]:
import nltk
nltk.download('stopwords')
tokenizer = ToktokTokenizer()

stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manoj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#Removing noise text

def strip_html(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

def remove_between_square_brackets(text):
  return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  return text

IMDB_df['review'] = IMDB_df['review'].apply(denoise_text)

C:\Users\manoj\AppData\Local\Temp\ipykernel_23848\3752247428.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [11]:
#remove special characters

def remove_special_characters(text, remove_digits=True):
  pattern = r'[^a-zA-Z0-9\s]'
  text = re.sub(pattern,'', text)
  return text

IMDB_df['review'] = IMDB_df['review'].apply(remove_special_characters)

In [12]:
# Removing stopwords

stop = set(stopwords.words('english'))
print(stop)

{'any', 'they', 'its', 's', 'who', 'of', 'an', 'whom', "aren't", 'very', 'your', 'haven', 'itself', 'too', 'when', 'ourselves', 'be', 'him', "hasn't", 'has', 'by', 'what', 'so', 'shan', 'hasn', 'same', 'about', 'during', "didn't", 'yours', "doesn't", 'down', 'such', 'that', 'isn', 'as', 'further', "wasn't", 'myself', 'on', 'y', 'to', 'had', 'himself', 'other', 'did', 'being', 'ours', 'them', 'once', 'll', 'only', 'own', 'mustn', "shouldn't", 'our', 'my', 'yourself', 'were', "don't", 'couldn', "weren't", 'you', 'then', 'both', 'shouldn', 'where', 'because', 'not', 'from', 'each', "won't", "you'll", 'those', 'theirs', 've', 'her', 'hers', 'for', 'if', 't', "mustn't", 'hadn', 'off', 'under', "wouldn't", 'won', 'their', 'now', 'few', 'but', 'me', "you're", 'here', 'will', 'out', 'he', 'am', 'which', "that'll", "she's", 'wasn', 'more', 'why', "hadn't", 'doesn', 'these', 'themselves', 'o', "isn't", 'don', 'doing', "you've", "shan't", 'while', 'weren', 'mightn', 'is', "mightn't", 'having', 's

In [13]:
def remove_stopwords(text, is_lower_case = False):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
    filtered_tokens = [token for token in tokens if token not in stopword_list ]

  else:
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list ]

  filtered_text = ' '.join(filtered_tokens)
  return filtered_text

IMDB_df['review'] = IMDB_df['review'].apply(remove_stopwords)

In [14]:
IMDB_df.head()

,review,sentiment
0,One reviewers mentioned watching 1 Oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,Basically theres family little boy Jake thinks...,negative
4,Petter Matteis Love Time Money visually stunni...,positive


In [15]:
# text stemming

def simple_stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

IMDB_df['review'] = IMDB_df['review'].apply(simple_stemmer)

In [16]:
IMDB_df['cleaned_review'] = IMDB_df['review']
IMDB_df.head()

,review,sentiment,cleaned_review
0,one review mention watch 1 oz episod youll hoo...,positive,one review mention watch 1 oz episod youll hoo...
1,wonder littl product film techniqu unassum old...,positive,wonder littl product film techniqu unassum old...
2,thought wonder way spend time hot summer weeke...,positive,thought wonder way spend time hot summer weeke...
3,basic there famili littl boy jake think there ...,negative,basic there famili littl boy jake think there ...
4,petter mattei love time money visual stun film...,positive,petter mattei love time money visual stun film...


In [17]:
#  Create X and y objects
X = IMDB_df['cleaned_review']
y = IMDB_df['sentiment']

In [18]:
label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)

print(y_encoded)

[1 1 1 ... 0 0 0]


In [19]:
# BoW object
bow_vectorizer = CountVectorizer(max_features=2500)

X_bow = bow_vectorizer.fit_transform(X)

In [20]:
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, y_encoded, test_size=0.2, random_state=0)

In [21]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB().fit(X_train_bow,y_train_bow)

y_pred_bow = model1.predict(X_test_bow)

In [22]:
# Display the confusion matrix
conf_matrix_bow = confusion_matrix(y_test_bow, y_pred_bow)
print("\nConfusion Matrix (BoW):")
print(conf_matrix_bow)


Confusion Matrix (BoW):
[[4233  802]
 [ 847 4118]]


#### Train Support Vector machine model

In [23]:
print(IMDB_df['review'].to_dict)

<bound method Series.to_dict of 0        one review mention watch 1 oz episod youll hoo...
1        wonder littl product film techniqu unassum old...
2        thought wonder way spend time hot summer weeke...
3        basic there famili littl boy jake think there ...
4        petter mattei love time money visual stun film...
                               ...                        
49995    thought movi right good job wasnt creativ orig...
49996    bad plot bad dialogu bad act idiot direct anno...
49997    cathol taught parochi elementari school nun ta...
49998    im go disagre previou comment side maltin one ...
49999    one expect star trek movi high art fan expect ...
Name: review, Length: 50000, dtype: object>


In [24]:
new_text = "This movie is best!"


cleaned_text = simple_stemmer(new_text)

new_text = bow_vectorizer.transform([cleaned_text])

print(new_text.shape)
prediction = model1.predict(new_text)

predicted_sentiment = label_encoder.inverse_transform(prediction)[0]

print(f"Original Text: {new_text}")
print(f"Cleaned Text: {cleaned_text}")
print(f"Predicted Sentiment: {predicted_sentiment}")

(1, 2500)
Original Text:   (0, 235)	1
  (0, 1471)	1
Cleaned Text: thi movi is best!
Predicted Sentiment: positive


In [25]:
producer1 = KafkaProducer(bootstrap_servers='localhost:9092')
producer1.send('test', b'Hello, Kafka!')


In [26]:
import time
import jsonpickle

for index, row in IMDB_df.head(10).iterrows():  
    new_text = row['review']
    cleaned_text = simple_stemmer(new_text)
    new_text = bow_vectorizer.transform([cleaned_text])

    #print(new_text.shape)
    prediction = model1.predict(new_text)
    predicted_sentiment = label_encoder.inverse_transform(prediction)[0]
    
    print(predicted_sentiment)
    #print(f"Original Text: {new_text}")
    print(f"Cleaned Text: {cleaned_text}")
    print(f"Predicted Sentiment: {predicted_sentiment}")
    
    message = {
    'review': new_text,
    'predicted_sentiment': predicted_sentiment
    }
    #print(message)
    producer.send('test',  row['review'])
    producer.send('test',  predicted_sentiment)
    time.sleep(5)



positive
Cleaned Text: one review mention watch 1 oz episod youll hook right exactli happen meth first thing struck oz brutal unflinch scene violenc set right word go trust show faint heart timid show pull punch regard drug sex violenc hardcor classic use wordit call oz nicknam given oswald maximum secur state penitentari focu mainli emerald citi experi section prison cell glass front face inward privaci high agenda em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti surreal couldnt say readi watch develop tast oz got accustom high level graphic violenc violenc injust crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison experi watch

In [27]:
import pytchat
import time

chat = pytchat.create(video_id = "sHwxI4FX6Jw")

while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.message}")
        
        review_text = c.message
        print(review_text)
        cleaned_text = simple_stemmer(review_text)
        new_text = row['review']
        cleaned_text = simple_stemmer(new_text)
        new_text = bow_vectorizer.transform([cleaned_text])

        #print(new_text.shape)
        prediction = model1.predict(new_text)
        predicted_sentiment = label_encoder.inverse_transform(prediction)[0]

        #print(f"Original Text: {new_text}")
        print(f"Cleaned Text: {cleaned_text}")
        print(f"Predicted Sentiment: {predicted_sentiment}")
        
        message = {
        'review': new_text,
        'predicted_sentiment': predicted_sentiment
        }
        print(message)
        producer.send('test', row['review'])
        producer.send('test', predicted_sentiment)
        time.sleep(5)
      

THANKS JÜRGEN
THANKS JÜRGEN
Cleaned Text: like origin gut wrench laughter like movi young old love movi hell even mom like itgreat camp
Predicted Sentiment: negative
{'review': <1x2500 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>, 'predicted_sentiment': 'negative'}
HUSAM WAS KLOPP OUT IM SURE HIS HAPPY
HUSAM WAS KLOPP OUT IM SURE HIS HAPPY
Cleaned Text: like origin gut wrench laughter like movi young old love movi hell even mom like itgreat camp
Predicted Sentiment: negative
{'review': <1x2500 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>, 'predicted_sentiment': 'negative'}
Xabi appointed too early
Xabi appointed too early
Cleaned Text: like origin gut wrench laughter like movi young old love movi hell even mom like itgreat camp
Predicted Sentiment: negative
{'review': <1x2500 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row